In [1]:
# given location (long & lat), find block id

import numpy as np
import matplotlib.path as mplPath

In [2]:
def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

In [3]:
def findBlock(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        z = mplPath.Path(np.array(zones.geometry[idx].exterior))
        if z.contains_point(np.array(p)):
            return zones['OBJECTID'][idx]
    return -1

In [4]:
def mapToZone(parts):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones("/Users/RAFAELLICHEN/Documents/CSC 59970/nyc-yellow-taxi-prediction/datasets/block-groups-polygons.geojson")
    for line in parts:
        if (line['pickup_longitude'] and line['pickup_latitude']):
            pickup_location  = geom.Point(proj(float(line['pickup_longitude']), float(line['pickup_latitude'])))
            pickup_block = findBlock(pickup_location, index, zones)
            if pickup_block >= 0:
                print (pickup_block)

In [5]:
import pandas as pd
data_pd = pd.read_csv("/Users/RAFAELLICHEN/Downloads/column_deleted_data_2012/yellow_tripdata_2012-01.csv")
data_pd.head()

,Unnamed: 0,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount,total_amount
0,0,2012-01-10 23:55:50,2012-01-11 00:03:39,1,1.7,-73.994693,40.725031,-73.975951,40.730781,1.00,8.90
1,1,2012-01-11 19:18:25,2012-01-11 19:26:10,1,1.1,-73.987955,40.752947,-73.994532,40.761040,0.00,7.60
2,2,2012-01-11 19:19:19,2012-01-11 19:48:15,2,18.0,-73.783093,40.648550,-73.996134,40.747624,10.06,60.36
3,3,2012-01-11 19:19:21,2012-01-11 19:27:00,1,1.7,-73.967515,40.758454,-73.956583,40.779903,1.00,9.40
4,4,2012-01-11 14:38:15,2012-01-11 14:43:51,1,1.2,-74.011315,40.711449,-74.002872,40.728131,0.00,6.20


In [10]:
# testing
mapToZone(data_pd.head(20).T.to_dict().values())

ImportError: No module named 'geopandas'